# Tutorial 16: Use Copilot to generate multilingual dialogues with SDialog


We will ask to Copilot or Cursor to create the code for us:

```plain
#fetch https://sdialog.readthedocs.io/en/latest/llm.txt

Write a Python code to generate dialog between an patient and a polite doctor using sdialog with multi-agent support, using Amazon bedrock call to Claude 3.5 sonnet. Both speakers should speak spanish.
```

The output:

In [ ]:
!pip install boto3 langchain-aws

In [ ]:
import os
import sdialog
from sdialog import Context 
from sdialog.agents import Agent
from sdialog.personas import Persona

AWS configuration with your usual method:

In [ ]:
os.environ["AWS_BEARER_TOKEN_BEDROCK"] = "XXX"

Configure sdialog to use your model:

In [ ]:
sdialog.config.llm("amazon:anthropic.claude-3-5-sonnet-20240620-v1:0", 
                   temperature=0.7, 
                   max_tokens=512,
                   region_name="us-east-1")

Define the characters:

In [ ]:
patient = Persona(
    name="María González",
    age=45,
    gender="female",
    role="paciente",
    background="María es una paciente de 45 años que consulta por dolores de cabeza recurrentes y fatiga. Trabaja en una oficina y tiene dos hijos adolescentes.",
    personality="María es educada, un poco ansiosa, y tiende a hacer muchas preguntas. Se preocupa fácilmente por su salud.",
    circumstances="Hizo una cita porque sus síntomas han persistido durante dos semanas y están afectando su trabajo.",
    rules="Debe ser respetuosa con el doctor y expresar sus preocupaciones claramente. Debe hablar en español.",
    language="Spanish"
)

doctor = Persona(
    name="Dr. Carlos Rodríguez",
    age=52,
    gender="male", 
    role="médico general",
    background="El Dr. Rodríguez es un médico general experimentado con 25 años de experiencia. Es conocido por su enfoque empático y paciencia con sus pacientes.",
    personality="Es muy profesional, empático, y se toma el tiempo para escuchar a sus pacientes. Explica las cosas de manera clara y tranquilizadora.",
    circumstances="Practica en su consultorio privado y tiene un enfoque holístico de la medicina.",
    rules="Debe ser cortés, profesional, hacer preguntas relevantes, y tranquilizar al paciente mientras permanece médicamente preciso. Debe hablar en español.",
    language="Spanish"
)

Create the consultation context:

In [ ]:
context = Context(
    location="Consultorio del Dr. Rodríguez",
    datetime="2024-01-15 14:30",
    environment="Un consultorio médico moderno y acogedor con una atmósfera tranquila y profesional",
    topics=["consulta médica", "dolores de cabeza", "fatiga", "diagnóstico"],
    goals=["establecer un diagnóstico", "tranquilizar al paciente", "proponer tratamiento"],
    constraints=["respetar la confidencialidad médica", "ser profesional", "limitar la consulta a 20 minutos"]
)

Create the agents with your model:

In [ ]:
agent_patient = Agent(
    persona=patient,
    name="María",
    context=context,
    first_utterance="Buenos días doctor, gracias por atenderme. Vine a verlo porque he tenido dolores de cabeza persistentes durante dos semanas, y me siento muy cansada.",
    dialogue_details="Consulta médica por síntomas persistentes",
    response_details="Respuestas naturales y respetuosas, expresando las preocupaciones del paciente en español"
)

agent_doctor = Agent(
    persona=doctor,
    name="Dr. Rodríguez", 
    context=context,
    dialogue_details="Consulta médica profesional",
    response_details="Respuestas médicas profesionales, empáticas y tranquilizadoras en español"
)

Generate the dialogue:

In [ ]:
dialogue = agent_patient.dialog_with(
    agent_doctor,
    context=context,
    max_turns=12,  # 6 échanges (12 tours au total)
    seed=42  # Pour la reproductibilité
)

dialogue.print()

print(f"\n=== Estadísticas del Diálogo ===")
print(f"Número de turnos: {len(dialogue)}")
print(f"Número de palabras: {dialogue.length('words')}")
print(f"Duración estimada: {dialogue.length('minutes')} minutos")
print(f"Modelo usado: {dialogue.model}")
print(f"Semilla: {dialogue.seed}")

Save the dialogue:

In [ ]:
dialogue.to_file("demo_dialog_doctor_patient_spanish.json")